# **Homework**: Data talks club data engineering zoomcamp Data loading workshop



In [28]:
#Install the dependencies
#%%capture
#!pip install dlt[duckdb]

## 1. Use a generator below to answer questions 1 and 2

In [3]:

def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)

1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979


### Question 1: What is the sum of the outputs of the generator for limit = 5?

#### Solution Q1:

In [4]:
from operator import add
from functools import reduce

### Question 1: What is the sum of the outputs of the generator for limit = 5?

generator_5 = square_root_generator(limit=5)

print(reduce(add, generator_5))        


8.382332347441762


### Question 2: What is the 13th number yielded?

#### Solution Q2:

In [5]:
### Question 2: What is the 13th number yielded?
generator_13 = square_root_generator(limit=13)
for _ in range(12):
    next(generator_13)
num_13 = next(generator_13)
print(num_13)

3.605551275463989


# 2. Append a generator to a table with existing data


Below you have 2 generators `people_1()` and `people_2()`. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**


In [26]:
import dlt
import duckdb

def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
   print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}


In [27]:
def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}

for person in people_2():
    print(person)

{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


### Question 3: After correctly appending the data, calculate the sum of all ages of people.

#### Solution Q3:

In [36]:
import dlt
import duckdb

people_gen_1 = people_1()
people_gen_2 = people_2()

# define the connection to load to. 
pipeline_people_1 = dlt.pipeline(pipeline_name="people_1_pipeline",
						destination='duckdb', 
						dataset_name='people_1_dataset')

# run the pipeline with the first generator, and capture the outcome
info = pipeline_people_1.run(people_gen_1, 
					table_name="users", 
					write_disposition="replace")
# print(info)

# run the pipeline with the second generator, write to the same table
info = pipeline_people_1.run(people_gen_2, 
					table_name="users", 
					write_disposition="append")
# print(info)

# show the outcome
conn = duckdb.connect(f"{pipeline_people_1.pipeline_name}.duckdb")

# check the tables
conn.sql(f"SET search_path = '{pipeline_people_1.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

print("\n\n\n Query results: sum of all ages of people")
sum_ages = conn.sql("""
SELECT SUM(age) as 'sum of ages'
FROM users
LIMIT 1
""").df()
display(sum_ages)

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ users               │
└─────────────────────┘




 Query results: sum of all ages of people


┌─────────────┐
│ sum of ages │
│   int128    │
├─────────────┤
│         353 │
└─────────────┘

# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.


### Question 4: Calculate the sum of ages of all the people loaded as described above.

#### Solution Q4:

In [42]:
people_gen_1 = people_1()
people_gen_2 = people_2()

# define the connection to load to. 
pipeline_people_1 = dlt.pipeline(pipeline_name="people_1_pipeline",
						destination='duckdb', 
						dataset_name='people_1_dataset')

# run the pipeline with the first generator, and capture the outcome
info = pipeline_people_1.run(people_gen_1, 
					table_name="users_merge", 
					write_disposition="replace")
# print(info)

# run the pipeline with the second generator, write to the same table
info = pipeline_people_1.run(people_gen_2, 
					table_name="users_merge", 
					write_disposition="merge",
                    primary_key="id")
# print(info)

# show the outcome
conn = duckdb.connect(f"{pipeline_people_1.pipeline_name}.duckdb")

# check the tables
conn.sql(f"SET search_path = '{pipeline_people_1.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))


print("\n\n\n Query results: sum of all ages of people")
sum_ages_merged = conn.sql("""
SELECT SUM(age) as 'sum of ages'
FROM users_merge
LIMIT 1
""").df()
display(sum_ages_merged)

print("\n\n\n View resulting table")
view_users_merged = conn.sql("""
SELECT id, name, age
FROM users_merge
ORDER BY id
""")
display(view_users_merged)

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ users               │
│ users_merge         │
└─────────────────────┘




 Query results: sum of all ages of people


,sum of ages
0,266.0


┌───────┬──────────┬───────┐
│  id   │   name   │  age  │
│ int64 │ varchar  │ int64 │
├───────┼──────────┼───────┤
│     1 │ Person_1 │    26 │
│     2 │ Person_2 │    27 │
│     3 │ Person_3 │    33 │
│     4 │ Person_4 │    34 │
│     5 │ Person_5 │    35 │
│     6 │ Person_6 │    36 │
│     7 │ Person_7 │    37 │
│     8 │ Person_8 │    38 │
└───────┴──────────┴───────┘